# Step 1: fine-tune LLM using top result from (fixed) ranker

In [2]:
# NB: amulet dropped some of the log output when this was run (?)
def launch():
    import os
    import StepOne
    import torch

    os.environ['MODEL_TYPE'] = 'xxl'
    augment = int(os.environ.get('AUGMENT', '4'))
    os.environ['BATCH_SIZE'] = '1'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepOne.step_one,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 4 max_iteration = 5 model_type = xxl *********
n              iter (since)      4 loss (since)       4 MAE (since) 4 MAE (dev) (since)      dt
8             0.000 (0.000)       0.374 (0.698)       0.500 (1.000)       0.000 (0.000)  22.1 s
16            0.000 (0.000)       0.331 (0.288)       0.250 (0.000)       0.000 (0.000)  43.5 s
32            0.000 (0.000)       0.374 (0.416)       0.219 (0.188)       0.000 (0.000)  1.41 m
64            0.000 (0.000)       0.640 (0.905)       0.297 (0.375)       0.000 (0.000)  2.76 m
128           0.000 (0.000)       0.654 (0.669)       0.328 (0.359)       0.000 (0.000)  5.57 m
256           0.000 (0.000)       0.635 (0.615)       0.297 (0.266)       0.000 (0.000)  11.1 m
512           0.000 (0.000)       0.553 (0.472)       0.258 (0.219)       0.000 (0.000)    22 m
1024          0.000 (0.000)       0.541 (0.529)       0.237 (0.217)       0.000 (0.000)  44.6 m
2048          0.000 (0.000)       0.532 (0.524)       0.238 (0.239)   

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def launch():
    import os
    import StepTwo
    import torch

    # NB: these are 80Gb gpu ram settings
    os.environ['MODEL_TYPE'] = 'xxl'
    augment = int(os.environ.get('AUGMENT', '1'))
    os.environ['BATCH_SIZE'] = '1'
    os.environ['LEARN_BATCH_SIZE'] = '4'
    os.environ['STEP1_ITER'] = '2_augment4'
    
    world_size = torch.cuda.device_count()
    torch.multiprocessing.spawn(StepTwo.step_two,
                                args=(world_size,),
                                nprocs=world_size,
                                join=True)
    
launch()

******** augment = 0 max_iteration = 5 model_type = xxl *********
n              iter (since)      4 loss (since)       4 MAE (since) 4 MAE (dev) (since)      nsamps (since)      dt
1             0.000 (0.000)       0.234 (0.234)       0.000 (0.000)       0.000 (0.000)      64.000 (64.000)  37.5 s
2             0.000 (0.000)       0.118 (0.001)       0.000 (0.000)       0.000 (0.000)      33.000 (2.000)    45 s
4             0.000 (0.000)       1.786 (3.455)       0.250 (0.500)       0.000 (0.000)      17.750 (2.500)  56.4 s
8             0.000 (0.000)       1.210 (0.634)       0.250 (0.250)       0.000 (0.000)      19.125 (20.500)  1.71 m
16            0.000 (0.000)       0.928 (0.647)       0.312 (0.375)       0.000 (0.000)      15.000 (10.875)  3.29 m
32            0.000 (0.000)       0.670 (0.412)       0.219 (0.125)       0.000 (0.000)      11.750 (8.500)  5.37 m
64            0.000 (0.000)       0.592 (0.514)       0.219 (0.219)       0.000 (0.000)       8.859 (5.969)  9.25 m
128